In [1]:
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt 
import networkx as nx 
import numpy as np
import pandas as pd 

from scipy import stats

Load in data as networkx object

In [2]:
import matplotlib

In [3]:
scerev = pd.read_excel("../206 Final Project/critical_network_data.xlsx", 
                       sheet_name=4, index_col=0)
# remove whitespace from ecoli index 
scerev.index = scerev.index.str.strip()
scerev_nx = nx.from_numpy_matrix(np.array(scerev))
ess_rxn_aer = pd.read_csv("essential_rxns.txt", error_bad_lines = False,
                          header=None)
# restrict essential reactions to those found in the adj matrix
ess_rxn_aer = pd.DataFrame(scerev[scerev.index.isin(ess_rxn_aer[0])].index)

/Users/rbbrown/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
scerev_nx

Panel a: log(bridging centrality)

In [5]:
def bridging_centrality(graph: nx.Graph) -> Dict:
    """
    Compute (modified) bridging centrality as defined in methods 
    section.
    
    :param graph: networkx (directed) graph 
    :return: Dict mapping nodes to briding centrality 
    """
    degrees_all = dict(nx.degree(graph))
    degree_in = dict(nx.DiGraph(graph).in_degree())
    degree_out = dict(nx.DiGraph(graph).out_degree())
    
    between_centr = nx.betweenness_centrality(graph)
    
    bc = {} # map node to bridging centrality 
    for rxn in degrees_all:
        if degree_in[rxn] == 0 or degree_out[rxn] == 0:
            bc[rxn] = 0
        else:
            numerator = 1/degrees_all[rxn]
            denominator = np.sum([1/degrees_all[x] for x in 
                                  graph.neighbors(rxn)])
            bc[rxn] = (numerator / denominator)*between_centr[rxn]
    return bc 

In [6]:
def prepare_graphing_data(metric: List, 
        essential: pd.DataFrame) -> List:
    """
    Count share of essential/non-essential reactions in each bin, percent 
    essential within a bin.
    
    :param hist_bins: List of (rounded) graph statistic values. 
    :param essential: DataFrame of essential reactions.
    :return: Essential reactions, nonessential reactions, % essential 
             within each bin 
    """
    # get essential reaction indices 
    ess_idx = np.where(scerev.index.isin(essential[0]))[0].tolist()
    
    # count *all* reactions in each bin
    # drop any inf's if there are any (an issue when we take log)
    all_rxn_per_bin = np.unique([x for x in metric if x != -np.inf], return_counts=True)
    
    # count *essential* reactions in each bin (including bins with 0 ess!)
    ess_only = np.array(metric)[ess_idx]
    ess_rxn_per_bin = [(ess_only == x).sum() 
                       for x in all_rxn_per_bin[0]]

    # get percent essential per bin 
    perc_ess_per_bin = [x/y for x,y in zip(ess_rxn_per_bin,
                                           all_rxn_per_bin[1])]
    # get count of non-essential per bin
    non_ess_per_bin = [x-y for x,y in zip(all_rxn_per_bin[1], 
                                          ess_rxn_per_bin)]

    return ess_rxn_per_bin, non_ess_per_bin, perc_ess_per_bin

Import pdb; pdb.set_trace()

In [7]:
from scipy.stats import linregress

In [35]:
def construct_plot(ess: List, non_ess: List, perc_ess: List,
                   x_axis: List, x_title: str, out_path: str) -> None:
    """
    Generate the combined histogram and scatterplot from the corresponding data. 
    
    :param ess: List of counts of essential reactions per bin. 
    :param non_ess: List of counts of non-essential reactions per bin. 
    :param perc_ess: List of percent of essential reactions per bin. 
    :param x_axis: List of x axis tick *labels* that match the paper directly. 
    :param x_title: x-axis title corresponding to graph metric. 
    :param out_path: Where to save figure. 
    """
    spacing = [x for x in range(len(x_axis))]
    fig, ax1 = plt.subplots()
    ax1.bar(spacing, ess, color="black")
    ax1.bar(spacing, non_ess, color="gray", bottom=ess)
    ax1.set_ylabel("Number of Reactions")
    plt.xticks(spacing, x_axis)
    plt.xlabel(x_title)

    ax2 = ax1.twinx()
    ax2.set_ylabel("% Essential")
    ax2.scatter(spacing, perc_ess, color="black")
    # line of best fit 
    m, b = np.polyfit(spacing, perc_ess, 1)
    ax2.plot(spacing, [(m*x) + b for x in spacing], color="black")

    # correlation
    res = stats.linregress(spacing, perc_ess)
    plt.plot([], [], ' ', label = f"r={round(np.corrcoef(perc_ess, spacing)[0,1],2)}, p-value ={round(res.pvalue,2)}, r-squared={res.rvalue**2:.6}")
    plt.legend(frameon=False, prop={'size': 12},bbox_to_anchor=(1.05, 1.0), loc='upper right')
    
    plt.savefig(out_path)
    plt.clf()

Panel a: log(bridging centrality)

In [36]:
bridge_centr = bridging_centrality(scerev_nx)
log10_bridge_centr = np.log10(list(bridge_centr.values()))
ess, non_ess, perc_ess = prepare_graphing_data([round(x*2)/2 if x != -np.inf else -np.inf for x in log10_bridge_centr], ess_rxn_aer)
construct_plot(ess[:-1], non_ess[:-1], perc_ess[:-1], [x/2 for x in range(len(ess)-1)], r'$log_{10}$(Bridging Centrality)', "../206 Final Project/panela2")

/var/folders/sb/27q12j591yxfmfcz9414bbbr0000gn/T/ipykernel_55581/430583142.py:2: RuntimeWarning: divide by zero encountered in log10
  log10_bridge_centr = np.log10(list(bridge_centr.values()))


<Figure size 432x288 with 0 Axes>

Panel b: log(betweenness centrality)

In [37]:
between_centr = list(nx.betweenness_centrality(scerev_nx).values())
log10_between_centr = np.log10(between_centr)

## shift to the right by the minimum positive log10 value 
min_pos_log10bc = np.min([x for x in log10_between_centr if x != -np.inf])
log10_between_centr = [x + np.abs(min_pos_log10bc) if x != -np.inf else -np.inf for x in log10_between_centr]
##

ess, non_ess, perc_ess = prepare_graphing_data([round(x*2)/2 if x != -np.inf else -np.inf for x in log10_between_centr], ess_rxn_aer)
construct_plot(ess, non_ess, perc_ess, [x/2 for x in range(len(ess))], r'$log_{10}$(Betweenness Centrality)', "../206 Final Project/panelb2")

/var/folders/sb/27q12j591yxfmfcz9414bbbr0000gn/T/ipykernel_55581/293653456.py:2: RuntimeWarning: divide by zero encountered in log10
  log10_between_centr = np.log10(between_centr)


<Figure size 432x288 with 0 Axes>

Panel c: clustering coefficient

In [38]:
clust_coef = list(nx.clustering(scerev_nx).values())
ess, non_ess, perc_ess = prepare_graphing_data([max(0.1, round(x,1)) for x in clust_coef], ess_rxn_aer)
construct_plot(ess, non_ess, perc_ess, [(x+1)/10 for x in range(len(ess))], "Clustering Coefficient", "../206 Final Project/panelc2")

<Figure size 432x288 with 0 Axes>

Panel d: log(degree)

In [39]:
from scipy.stats import linregress

In [40]:
degree = list(dict(nx.degree(scerev_nx)).values())
log10_degree = np.log10(degree)
ess, non_ess, perc_ess = prepare_graphing_data([round(x*4)/4 for x in log10_degree], ess_rxn_aer)
construct_plot(ess, non_ess, perc_ess, [x/4 for x in range(len(ess))], r'$log_{10}$(Degree)', "../206 Final Project/paneld2")
plt.show()

<Figure size 432x288 with 0 Axes>